In [ ]:
pip install tika

In [51]:
# 의사록 Crawling

import requests
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd

start_page = 1
current_page = start_page
end_page = 31

# 32페이지는 링크따와서 수작업
host = 'http://www.bok.or.kr/'
result_df = pd.DataFrame(columns=['date', 'text'])

while True:

    URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(current_page)
    r = requests.get(URL)
    sp = BeautifulSoup(r.text, 'html.parser')
    results = sp.select('div.fileGoupBox ul')
    
    for i, result in enumerate(results):
        pdf_date = sp.select('span.date')[i].text[3:]
        
        pdf_temp = requests.get(host + result.select('li')[1].select('a')[0].attrs['href'])

        with open('bok.pdf', 'wb') as f:
            f.write(pdf_temp.content)
        
        parsed = parser.from_file('bok.pdf')
        temp_text = str(parsed['content']).strip()
        temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
        result_df = pd.concat([result_df, temp_df])

    if current_page == end_page:
        result_df.to_csv('datas.csv')
        break
    current_page += 1


In [113]:
# 32 페이지 수작업 부분
URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=32'
r = requests.get(URL)
sp = BeautifulSoup(r.text, 'html.parser')
results = sp.select('div.fileGoupBox ul')

for i, result in enumerate(results):
    pdf_date = sp.select('span.date')[i].text[3:]

    pdf_temp = requests.get(host + result.select('li')[1].select('a')[0].attrs['href'])

    with open('bok.pdf', 'wb') as f:
        f.write(pdf_temp.content)

    parsed = parser.from_file('bok.pdf')
    temp_text = str(parsed['content']).strip()
    temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
    test_df = pd.concat([test_df, temp_df])

In [111]:
URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=32'
r = requests.get(URL)
sp = BeautifulSoup(r.text, 'html.parser')
results = sp.select('div.fileGoupBox ul')

In [78]:
results = sp.select('div.fileLink')

In [88]:
results[1].select('a')[0].attrs['href']

'/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085858&fileSn=1'

In [74]:
pdf_date = sp.select('span.date')[7].text[3:]

In [101]:
for i, result in enumerate(results):
    if i==0:
        continue
    
    pdf_date = sp.select('span.date')[i].text[3:]

    pdf_temp = requests.get(host + results[i].select('a')[0].attrs['href'])

    with open('bok.hwp', 'wb') as f:
        f.write(pdf_temp.content)

    parsed = parser.from_file('bok.hwp')
    temp_text = str(parsed['content']).strip()
    temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
    test_df = pd.concat([test_df, temp_df])



In [116]:
test_df.to_csv('bok_data.csv',encoding='utf-8')

In [2]:
import pandas as pd
bok_df = pd.read_csv('bok_datas.csv')

In [3]:
bok_df = bok_df[['date', 'text']]

In [62]:
bok_df.iloc[:10] # 앞부분 다름 index 299 양식 다름 - 여러 개 있음 index 300~ 양식 다름

,date,text
0,2020.07.10,2020년도 제14차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
1,2020.06.30,2020년도 제13차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
2,2020.06.16,2020년도 제12차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
3,2020.06.02,2020년도 제11차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
4,2020.05.06,2020년도 제9차\n\n금융통화위원회(임시) 의사록\n\n한 국 은 행\n\n\n...
5,2020.04.28,2020년도 제8차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n...
6,2020.04.14,2020년도 제7차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n...
7,2020.03.31,2020년도 제6차\n\n금융통화위원회(임시) 의사록\n\n한 국 은 행\n\n\n...
8,2020.03.31,2020년도 제5차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n...
9,2020.03.17,2020년도 제4차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n...


In [129]:
# bok_df.iloc[0]['text']

In [130]:
# bok_df.iloc[299]['text']

In [131]:
# bok_df.iloc[300]['text']

In [112]:
import re
import numpy as np
def section_split(bok_text):
    bok_text = bok_text.replace('\n', ' ')
    s2_ls = []
    s3_ls = []
    section2 = re.finditer('(?<=원 토의내용)((?!심의결과).)*', bok_text)
    for s2 in section2:
        s2_ls.append(s2.group())
    section3 = re.finditer('(?<=심의결과)[^１]*', bok_text)
    for s3 in section3:
        s3_ls.append(s3.group())
    return ' '.join(s2_ls), ' '.join(s3_ls)


split_text = list(map(section_split, bok_df['text']))

TypeError: iteration over a 0-d array

In [117]:
len(split_text)

310

In [118]:
s2_ls = []
s3_ls = []

for doc in split_text:
    s2_ls.append(doc[0])
    s3_ls.append(doc[1])

In [121]:
bok_df['section2'] = s2_ls

In [122]:
bok_df['section3'] = s3_ls

In [124]:
bok_df[['date', 'section2','section3']].to_csv('bok_split_section.csv', encoding='utf-8')

In [125]:
bok_df.isnull().sum()

date        0
text        0
section2    0
section3    0
dtype: int64

In [127]:
bok_df.iloc[30]['section2']

''

In [132]:
bok_df.iloc[30]['text']

'2018년도 제23차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n\n- 1 -\n\n1. 일 자 2018년 12월 6일(목)\n\n2. 장 소 금융통화위원회 회의실\n\n3. 출석위원 이 주 열 의 장(총재)\n\n이 일 형 위 원\n\n조 동 철 위 원\n\n고 승 범 위 원\n\n신 인 석 위 원\n\n윤 면 식 위 원(부총재)\n\n임 지 원 위 원\n\n4. 결석위원 없 음\n\n5. 참 여 자 장 호 현 감 사 임 형 준 부총재보\n\n허 진 호 부총재보 신 호 순 부총재보\n\n정 규 일 부총재보 서 봉 국 외자운용원장\n\n이 환 석 조사국장 손 욱 경제연구원장\n\n신 운 금융안정국장 박 종 석 통화정책국장\n\n이 상 형 금융시장국장 이 승 헌 국제국장\n\n장 정 석 금융통화위원회실장 김 현 기 공보관\n\n성 광 진 의사팀장\n\n6. 회의경과\n\n<의안 제39호 - 대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령\n\n제9조제3항제1호의 연체이자율에 관한 규정 폐지규정(안)>\n\n(１) 담당 부총재보가 대부업 등의 등록 및 금융이용자 보호에 관한 법률 시\n\n행령 개정으로 한국은행법 제11조에 따른 금융기관에 대한 연체이자율 상한을\n\n금융위원회에서 정하게 됨에 따라 관련 규정을 폐지하고자 한다는 내용의 제안설\n\n명을 하였음.\n\n(２) 심의결과\n\n원안대로 가결\n\n\n\n- 2 -\n\n의결사항\n\n대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령 제9조제3항\n\n제1호의 연체이자율에 관한 규정 을 붙임과 같이 폐지한다.\n\n<붙임> 대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령 제9조\n\n제3항제1호의 연체이자율에 관한 규정 폐지규정(안)'

In [1]:
import pandas as pd
bok_datas = pd.read_csv('bok_split_section.csv',encoding='utf-8')

In [4]:
bok_datas = bok_datas[['date', 'section2', 'section3']]

In [13]:
test_str = bok_datas['section2'][0]

In [9]:
# Section 2
import re
re.sub('- [0-9] -', '', test_str) # 페이지번호 삭제
re.sub('\([３]\)', '', test_str)

# Section 3

\(생략\)
\<붙임\>
개정\(안\)
원안대로 가결
의결사항
test_str.replace('  ', '')
# 스페이스바 2개 이상 앞에 .이면 한칸 띄우고
# 스페이스바 2개 이상 앞에 . 아니면 합치기


'  관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용      을 다음과 같이 보고하였음.  먼저 금번 보고서에서는 코로나19 확산에 따른 유례없는 금융·실물 충격이 금  융시스템에 미치는 영향을 점검하는 한편, 국내외 정책대응 등의 효과를 고려하  여 금융안정 상황을 종합적으로 평가하였음.  다음으로 6월 11일 위원협의회에서 논의된 내용은 다음과 같음.  여러 위원들은 금번 보고서가 코로나19 확산 이후 금융불안 우려가 높아진  상황에서 금융안정과 관련된 다양한 이슈를 균형있게 다루고 있다고 평가하였음.  일부 위원은 코로나19 확산 이후 금융시장 상황이 빠르게 변하고 있는 점을  감안하여 입수 가능한 통계를 최대한 활용하여 최근의 움직임을 반영할 필요가  있다는 의견을 제시하였음.  다른 일부 위원은 일부 중복되는 부분을 정비하고 보고서의 주요 메시지를  보다 분명하게 나타낼 필요가 있다는 견해를 밝혔음.  일부 위원은 ‘금융안정 상황’ 및 ‘복원력’과 관련하여, 기업신용 상황을 분석하  는 데 이용된 표본기업이 기업경영분석 등에 이용되는 표본기업과 서로 달라 독  자들의 혼동을 야기할 수 있으므로 지표해석 시 유의사항을 좀 더 자세히 기술할  필요가 있다는 의견을 나타내었음.  다른 일부 위원은 기업신용의 증가 원인을 수요와 공급 측면으로 나누어 기  술하는 것이 좋겠다는 견해를 나타내었음.  이와 관련하여 일부 위원은 최근 기업대출 증가와 관련된 위험요인을 언급할  필요가 있다는 의견을 제시하였음.  또한 일부 위원은 기업의 재무건전성 분석이 지난해까지의 통계에 기반하고  있다고 언급하면서, 코로나19 확산 이후의 상황도 함께 기술할 필요가 있다는 견  해를 밝혔음.  아울러 일부 위원은 회사채, CP 등의 순발행 상황, 채권시장 전망 등에 관해  기술한 부분에 당행과 정부의 정책대응 내용을 추가할 필요가 있다는 의견을 제  시하였음.      한편 일부 위원은 부동산금융 익스포저(exposure)의 동향을 좀 더 상세히 설 

In [12]:
test_str

'  원안대로 가결  의결사항  한국은행법 제96조 제1항에 의거 거시 금융안정상황에 대한 평가보고서를  붙임과 같이 작성하여 국회에 제출할 것을 의결한다.  <붙임> 금융안정보고서(2020년 6월)(생략)  <의안 제29호 –「지급결제제도 운영･관리규정」 개정(안)>  (   원안대로 가결  의결사항  ｢지급결제제도 운영‧관리규정｣을 붙임과 같이 개정한다  <붙임> ｢지급결제제도 운영·관리규정｣개정(안)(생략)'

In [16]:
test_str.replace('.', '. ').replace('  ', '')

'관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용- 2 -을 다음과 같이 보고하였음. 먼저 금번 보고서에서는 코로나19 확산에 따른 유례없는 금융·실물 충격이 금융시스템에 미치는 영향을 점검하는 한편, 국내외 정책대응 등의 효과를 고려하여 금융안정 상황을 종합적으로 평가하였음. 다음으로 6월 11일 위원협의회에서 논의된 내용은 다음과 같음. 여러 위원들은 금번 보고서가 코로나19 확산 이후 금융불안 우려가 높아진상황에서 금융안정과 관련된 다양한 이슈를 균형있게 다루고 있다고 평가하였음. 일부 위원은 코로나19 확산 이후 금융시장 상황이 빠르게 변하고 있는 점을감안하여 입수 가능한 통계를 최대한 활용하여 최근의 움직임을 반영할 필요가있다는 의견을 제시하였음. 다른 일부 위원은 일부 중복되는 부분을 정비하고 보고서의 주요 메시지를보다 분명하게 나타낼 필요가 있다는 견해를 밝혔음. 일부 위원은 ‘금융안정 상황’ 및 ‘복원력’과 관련하여, 기업신용 상황을 분석하는 데 이용된 표본기업이 기업경영분석 등에 이용되는 표본기업과 서로 달라 독자들의 혼동을 야기할 수 있으므로 지표해석 시 유의사항을 좀 더 자세히 기술할필요가 있다는 의견을 나타내었음. 다른 일부 위원은 기업신용의 증가 원인을 수요와 공급 측면으로 나누어 기술하는 것이 좋겠다는 견해를 나타내었음. 이와 관련하여 일부 위원은 최근 기업대출 증가와 관련된 위험요인을 언급할필요가 있다는 의견을 제시하였음. 또한 일부 위원은 기업의 재무건전성 분석이 지난해까지의 통계에 기반하고있다고 언급하면서, 코로나19 확산 이후의 상황도 함께 기술할 필요가 있다는 견해를 밝혔음. 아울러 일부 위원은 회사채, CP 등의 순발행 상황, 채권시장 전망 등에 관해기술한 부분에 당행과 정부의 정책대응 내용을 추가할 필요가 있다는 의견을 제시하였음. - 3 -한편 일부 위원은 부동산금융 익스포저(exposure)의 동향을 좀 더 상세히 설명할 필요가 있다는 견해를 제시하면서, 앞으로 보다 다양한 분석지표를 추가하는 

In [18]:
test_str

'  관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용    - 2 -  을 다음과 같이 보고하였음.  먼저 금번 보고서에서는 코로나19 확산에 따른 유례없는 금융·실물 충격이 금  융시스템에 미치는 영향을 점검하는 한편, 국내외 정책대응 등의 효과를 고려하  여 금융안정 상황을 종합적으로 평가하였음.  다음으로 6월 11일 위원협의회에서 논의된 내용은 다음과 같음.  여러 위원들은 금번 보고서가 코로나19 확산 이후 금융불안 우려가 높아진  상황에서 금융안정과 관련된 다양한 이슈를 균형있게 다루고 있다고 평가하였음.  일부 위원은 코로나19 확산 이후 금융시장 상황이 빠르게 변하고 있는 점을  감안하여 입수 가능한 통계를 최대한 활용하여 최근의 움직임을 반영할 필요가  있다는 의견을 제시하였음.  다른 일부 위원은 일부 중복되는 부분을 정비하고 보고서의 주요 메시지를  보다 분명하게 나타낼 필요가 있다는 견해를 밝혔음.  일부 위원은 ‘금융안정 상황’ 및 ‘복원력’과 관련하여, 기업신용 상황을 분석하  는 데 이용된 표본기업이 기업경영분석 등에 이용되는 표본기업과 서로 달라 독  자들의 혼동을 야기할 수 있으므로 지표해석 시 유의사항을 좀 더 자세히 기술할  필요가 있다는 의견을 나타내었음.  다른 일부 위원은 기업신용의 증가 원인을 수요와 공급 측면으로 나누어 기  술하는 것이 좋겠다는 견해를 나타내었음.  이와 관련하여 일부 위원은 최근 기업대출 증가와 관련된 위험요인을 언급할  필요가 있다는 의견을 제시하였음.  또한 일부 위원은 기업의 재무건전성 분석이 지난해까지의 통계에 기반하고  있다고 언급하면서, 코로나19 확산 이후의 상황도 함께 기술할 필요가 있다는 견  해를 밝혔음.  아울러 일부 위원은 회사채, CP 등의 순발행 상황, 채권시장 전망 등에 관해  기술한 부분에 당행과 정부의 정책대응 내용을 추가할 필요가 있다는 의견을 제  시하였음.    - 3 -  한편 일부 위원은 부동산금융 익스포저(exposure)의 동향을 

NameError: name 'dflkajsdlfjk' is not defined